In [1]:
import pinyin

In [2]:
chinese_dataset = 'article_9k.txt'
CHINESE_CHARATERS = open(chinese_dataset, encoding='utf8').read()


In [3]:
def chinese2pingying(word):
    return pinyin.get(word, format='strip', delimiter=' ')

In [4]:
pingying = chinese2pingying(CHINESE_CHARATERS)

In [5]:
pingying[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [6]:
import re

In [7]:
pingying = re.findall('[a-z]+', pingying)

In [8]:
from collections import Counter

In [9]:
pingying = Counter(pingying)

In [10]:
def edit_1(word):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    
    insert = [word[:i] + w + word[i:] for i in range(len(word) + 1) for w in alphabet]
    delete = [word[:i] + word[i + 1:] for i in range(len(word))]
    transposes = [word[:i] + word[i + 1] + word[i] + word[i + 2:] for i in range(len(word) - 1)]
    replaces = [word[:i] + w + word[i+1:] for i in range(len(word)) for w in alphabet]
    return set(insert + delete + transposes + replaces)

In [11]:
len(edit_1('pinyin'))

338

In [12]:
def known(words):
    return [word for word in words if word in pingying]

In [13]:
def correct(word):
    words = known([word]) or known(edit_1(word)) or known([e2 for e1 in edit_1(word) for e2 in edit_1(e1)]) or [word]
    return sorted(words, key=pingying.get)[-1]

In [14]:
def correct_sequence_pinyin(text_pingyin):
    return ' '.join(map(correct, text_pingyin.split()))

In [15]:
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

'wo xiang shang qing hua da xue'

In [16]:
from functools import lru_cache

In [113]:
p = 'woxiang'

In [118]:
def split(p):
    status_list = []
    
    status = [0 for _ in range(len(p) - 1)]
    
    def _change_status(status, num):
        if num > 0:
            num -= 1
            indexes = [i for i, _ in enumerate(status) if _ is 0]
            
            for index in indexes:
                _status = status.copy()
                _status[index] = 1
                yield (_status, num)
        else:
            yield (status, num)
            
    def _reduce_num_to_0(status, num):
        heap = []
        if num > 0:
            heap.append((status, num))
        
        while heap:
            status, num = heap.pop()
            num -= 1
            for status, num in _change_status(status, num):
                if num:
                    heap.append((status, num))
                else:
                    yield tuple(status)
        
    for num in range(1, len(p)):
        for s in _reduce_num_to_0(status, num):
            if s not in status_list:
                status_list.append(s)
                yield s


In [119]:
list(split(p))

1
2
3
4
5
6


[(0, 0, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, 0),
 (0, 0, 1, 0, 0, 0),
 (0, 0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 1),
 (1, 0, 0, 0, 0, 1),
 (0, 1, 0, 0, 0, 1),
 (0, 0, 1, 0, 0, 1),
 (0, 0, 0, 1, 0, 1),
 (0, 0, 0, 0, 1, 1),
 (1, 0, 0, 0, 1, 0),
 (0, 1, 0, 0, 1, 0),
 (0, 0, 1, 0, 1, 0),
 (0, 0, 0, 1, 1, 0),
 (1, 0, 0, 1, 0, 0),
 (0, 1, 0, 1, 0, 0),
 (0, 0, 1, 1, 0, 0),
 (1, 0, 1, 0, 0, 0),
 (0, 1, 1, 0, 0, 0),
 (1, 1, 0, 0, 0, 0),
 (1, 0, 0, 0, 1, 1),
 (0, 1, 0, 0, 1, 1),
 (0, 0, 1, 0, 1, 1),
 (0, 0, 0, 1, 1, 1),
 (1, 0, 0, 1, 0, 1),
 (0, 1, 0, 1, 0, 1),
 (0, 0, 1, 1, 0, 1),
 (1, 0, 1, 0, 0, 1),
 (0, 1, 1, 0, 0, 1),
 (1, 1, 0, 0, 0, 1),
 (1, 0, 0, 1, 1, 0),
 (0, 1, 0, 1, 1, 0),
 (0, 0, 1, 1, 1, 0),
 (1, 0, 1, 0, 1, 0),
 (0, 1, 1, 0, 1, 0),
 (1, 1, 0, 0, 1, 0),
 (1, 0, 1, 1, 0, 0),
 (0, 1, 1, 1, 0, 0),
 (1, 1, 0, 1, 0, 0),
 (1, 1, 1, 0, 0, 0)]

In [74]:
p_list = [el for el in p]
probility = []
all = sum([el for el in pingying.values()])

for cut in split(p):
    cut = [i for i, s in enumerate(cut) if s]
    p_temp = p_list.copy()
    for index in cut[::-1]:
        p_temp.insert(index, ' ')
    p_temp = ''.join(p_temp)
    words = []
    p_probility = 1
    for word in p_temp.split(' '):
        word = correct(word)
        words.append(word)
        if len(word) == 1:
            p_probility = 0
        p_probility *= pingying.get(word, 0) / all
#     if p_probility:
#         print(p_probility,words, 1/len(words))
    p_probility = p_probility ** (1/len(words))
    if p_probility:
        probility.append((' '.join(words), p_probility))
probility = list(set(probility))
probility.sort(key=lambda x: x[1], reverse=True)

In [75]:
probility

[('xiang', 0.006313906270151132),
 ('wo xiang', 0.0036199703703819405),
 ('wo xi ang', 0.0011289484676427332),
 ('wei ang', 0.001019031574775807)]

In [ ]:
p = 'wobuxianshanqinhuadaxue'

In [126]:
def split(p):
    result_dict = {}
    
    status_list = []
    
    status = tuple([0 for _ in range(len(p) - 1)])
    
#     @lru_cache(2**10)
    def _change_status(status):
        status = list(status)
        indexes = [i for i, _ in enumerate(status) if _ is 0]

        for index in indexes:
            _status = status.copy()
            _status[index] = 1
            yield tuple(_status)
            
    def _reduce_num_to_0(status, num):
        heap = []
        if num > 0:
            heap.append((status, num))
        while heap:
            status, num = heap.pop()
            num -= 1
            if status not in result_dict:
                for s in _change_status(status):
                    if num:
                        heap.append((s, num))
                    else:
                        result_dict.setdefault('_' + str(status), []).append(s)
                        yield s
                if '_' + str(status) in result_dict:
                    result_dict[status] = result_dict['_' + str(status)]
                    del result_dict['_' + str(status)]
                
            else:
                for s in result_dict[status]:
                    if num:
                        heap.append((s, num))
                    else:
                        result_dict.setdefault('_' + str(status), []).append(s)
                        yield s
                if '_' + str(status) in result_dict:
                    result_dict[status] = result_dict['_' + str(status)]
                    del result_dict['_' + str(status)]
    for num in range(1, len(p)):
        for s in _reduce_num_to_0(status, num):
            if s not in status_list:
                status_list.append(s)
                yield s


In [128]:
list((split('woxiangsd')))

[(1, 0, 0, 0, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, 0, 0, 0, 0, 0),
 (0, 0, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 0),
 (0, 0, 0, 0, 0, 1, 0, 0),
 (0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 0, 1),
 (1, 0, 0, 0, 0, 0, 0, 1),
 (0, 1, 0, 0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0, 0, 0, 1),
 (0, 0, 0, 1, 0, 0, 0, 1),
 (0, 0, 0, 0, 1, 0, 0, 1),
 (0, 0, 0, 0, 0, 1, 0, 1),
 (0, 0, 0, 0, 0, 0, 1, 1),
 (1, 0, 0, 0, 0, 0, 1, 0),
 (0, 1, 0, 0, 0, 0, 1, 0),
 (0, 0, 1, 0, 0, 0, 1, 0),
 (0, 0, 0, 1, 0, 0, 1, 0),
 (0, 0, 0, 0, 1, 0, 1, 0),
 (0, 0, 0, 0, 0, 1, 1, 0),
 (1, 0, 0, 0, 0, 1, 0, 0),
 (0, 1, 0, 0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0, 1, 0, 0),
 (0, 0, 0, 1, 0, 1, 0, 0),
 (0, 0, 0, 0, 1, 1, 0, 0),
 (1, 0, 0, 0, 1, 0, 0, 0),
 (0, 1, 0, 0, 1, 0, 0, 0),
 (0, 0, 1, 0, 1, 0, 0, 0),
 (0, 0, 0, 1, 1, 0, 0, 0),
 (1, 0, 0, 1, 0, 0, 0, 0),
 (0, 1, 0, 1, 0, 0, 0, 0),
 (0, 0, 1, 1, 0, 0, 0, 0),
 (1, 0, 1, 0, 0, 0, 0, 0),
 (0, 1, 1, 0, 0, 0, 0, 0),
 (1, 1, 0, 0, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, 0, 1, 1),
 